In [26]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus


In [27]:
# Credenciales base 1
user = "postgres"
password = "admin123"   # tu contraseña real
host = "localhost"
port = "5432"
database = "counterfeit"

In [28]:



# Codificar la contraseña para URL
password_encoded = quote_plus(password)

# Crear motor de conexión
engine = create_engine(
    f"postgresql+psycopg2://{user}:{password_encoded}@{host}:{port}/{database}"
)

# Leer CSV
df = pd.read_csv("counterfeit_transactions.csv")

# Subir a PostgreSQL
df.to_sql("counterfeit_transactions", engine, if_exists="replace", index=False)

print("✅ CSV cargado correctamente en PostgreSQL")


✅ CSV cargado correctamente en PostgreSQL


# ## 1) Carga de datos
# Intentamos leer desde **PostgreSQL** (tabla `counterfeit_transactions`).  


In [29]:
def cargar_transacciones(engine):
    if engine is not None:
        try:
            df = pd.read_sql("SELECT * FROM counterfeit_transactions", engine)
            print(f"Leído desde PostgreSQL: {df.shape}")
            return df
        except Exception as e:
            print(f"⚠️ No se pudo leer desde PostgreSQL: {e}")
    # fallback CSV
    df = pd.read_csv(CSV_PATH, low_memory=False)
    print(f"Leído desde CSV: {df.shape}")
    return df

df_transacciones = cargar_transacciones(engine)

# Normalizar tipos útiles
# Intento de parse de fecha si existe columna transaccional temporal
for col in df_transacciones.columns:
    if any(tok in col.lower() for tok in ["date", "time", "timestamp", "dt"]):
        df_transacciones[col] = pd.to_datetime(df_transacciones[col], errors="coerce")

# Campos derivados comunes
if "unit_price" in df_transacciones.columns and "quantity" in df_transacciones.columns:
    df_transacciones["monto_calculado"] = df_transacciones["unit_price"] * df_transacciones["quantity"]
if "total_amount" in df_transacciones.columns and "shipping_cost" in df_transacciones.columns:
    df_transacciones["monto_total_con_envio"] = df_transacciones["total_amount"] + df_transacciones["shipping_cost"]


Leído desde PostgreSQL: (3000, 20)


In [30]:
# Normalizar tipos útiles
# Intento de parse de fecha si existe columna transaccional temporal
for col in df_transacciones.columns:
    if any(tok in col.lower() for tok in ["date", "time", "timestamp", "dt"]):
        df_transacciones[col] = pd.to_datetime(df_transacciones[col], errors="coerce")

# Campos derivados comunes
if "unit_price" in df_transacciones.columns and "quantity" in df_transacciones.columns:
    df_transacciones["monto_calculado"] = df_transacciones["unit_price"] * df_transacciones["quantity"]
if "total_amount" in df_transacciones.columns and "shipping_cost" in df_transacciones.columns:
    df_transacciones["monto_total_con_envio"] = df_transacciones["total_amount"] + df_transacciones["shipping_cost"]

In [32]:
# ## 2) Exploración – `df_transacciones`
# **Entender columnas**: tipos, muestra, estadísticas y categorías.
# `head()` para ver primeras filas
# `info()` para tipos y nulos
# `describe()` para medidas básicas
# `value_counts()` en variables clave (método logístico, método de pago, banderas)

In [33]:
display(df_transacciones.head(10))
display(df_transacciones.info())
display(df_transacciones.describe(include="all"))

cols_categoricas = [c for c in df_transacciones.columns if df_transacciones[c].dtype == "object"]
for c in [x for x in cols_categoricas if x in ["payment_method","shipping_speed","customer_location"]]:
    print(f"\nDistribución de {c}:")
    display(df_transacciones[c].value_counts().head(10))

,transaction_id,customer_id,transaction_date,customer_age,customer_location,quantity,unit_price,total_amount,payment_method,shipping_speed,...,discount_percentage,shipping_cost,delivery_time_days,refund_requested,velocity_flag,geolocation_mismatch,device_fingerprint_new,involves_counterfeit,monto_calculado,monto_total_con_envio
0,TXN_957334,CUST_11907,2024-10-12 03:52:13,22,JP,1,199.66,199.66,Debit Card,Express,...,0.0,5.39,1970-01-01 00:00:00.000000026,False,False,False,False,False,199.66,205.05
1,TXN_246397,CUST_27641,2024-08-18 00:36:57,62,DE,4,116.01,464.03,PayPal,Express,...,0.0,8.18,1970-01-01 00:00:00.000000025,False,False,False,False,False,464.04,472.21
2,TXN_403072,CUST_78628,2024-08-19 22:21:30,75,DE,1,42.04,42.04,Credit Card,Priority,...,0.0,8.61,1970-01-01 00:00:00.000000027,False,False,False,False,False,42.04,50.65
3,TXN_848560,CUST_98579,2025-04-16 18:49:39,56,BR,3,147.69,443.08,Credit Card,Standard,...,48.7,17.00,1970-01-01 00:00:00.000000026,False,False,False,False,False,443.07,460.08
4,TXN_270817,CUST_67519,2024-10-06 13:30:52,19,IN,4,40.70,162.80,PayPal,Priority,...,0.0,20.53,1970-01-01 00:00:00.000000001,False,False,True,False,False,162.80,183.33
5,TXN_235574,CUST_76740,2024-07-14 20:02:30,30,IN,3,298.04,894.11,Debit Card,Express,...,40.5,17.06,1970-01-01 00:00:00.000000004,False,False,False,False,False,894.12,911.17
6,TXN_539896,CUST_25980,2025-03-22 14:34:20,67,US,1,52.95,52.95,PayPal,Express,...,30.4,8.19,1970-01-01 00:00:00.000000015,False,False,False,False,False,52.95,61.14
7,TXN_831689,CUST_71988,2024-09-28 05:25:30,25,IN,1,81.40,81.40,Credit Card,Express,...,39.0,12.74,1970-01-01 00:00:00.000000014,False,False,False,False,False,81.40,94.14
8,TXN_743257,CUST_72363,2024-09-27 13:10:51,66,US,1,79.57,79.57,PayPal,Standard,...,0.0,7.93,1970-01-01 00:00:00.000000013,False,False,False,False,False,79.57,87.50
9,TXN_379872,CUST_86291,2024-10-31 01:46:18,78,IN,1,193.72,193.72,PayPal,Standard,...,27.6,21.37,1970-01-01 00:00:00.000000019,False,False,False,False,False,193.72,215.09


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           3000 non-null   object        
 1   customer_id              3000 non-null   object        
 2   transaction_date         3000 non-null   datetime64[ns]
 3   customer_age             3000 non-null   int64         
 4   customer_location        3000 non-null   object        
 5   quantity                 3000 non-null   int64         
 6   unit_price               3000 non-null   float64       
 7   total_amount             3000 non-null   float64       
 8   payment_method           3000 non-null   object        
 9   shipping_speed           3000 non-null   object        
 10  customer_history_orders  3000 non-null   int64         
 11  discount_applied         3000 non-null   bool          
 12  discount_percentage      3000 non-

None

,transaction_id,customer_id,transaction_date,customer_age,customer_location,quantity,unit_price,total_amount,payment_method,shipping_speed,...,discount_percentage,shipping_cost,delivery_time_days,refund_requested,velocity_flag,geolocation_mismatch,device_fingerprint_new,involves_counterfeit,monto_calculado,monto_total_con_envio
count,3000,3000,3000,3000.000000,3000,3000.000000,3000.000000,3000.000000,3000,3000,...,3000.000000,3000.000000,3000,3000,3000,3000,3000,3000,3000.000000,3000.000000
unique,2997,2953,NaN,NaN,10,NaN,NaN,NaN,6,4,...,NaN,NaN,NaN,2,2,2,2,2,NaN,NaN
top,TXN_413571,CUST_23163,NaN,NaN,GB,NaN,NaN,NaN,PayPal,Standard,...,NaN,NaN,NaN,False,False,False,False,False,NaN,NaN
freq,2,3,NaN,NaN,325,NaN,NaN,NaN,771,1099,...,NaN,NaN,NaN,2460,2678,2515,2431,2267,NaN,NaN
mean,NaN,NaN,2025-01-04 01:09:03.542666496,48.665667,NaN,4.372333,133.781473,431.415573,NaN,NaN,...,8.573100,14.874780,1970-01-01 00:00:00.000000015,NaN,NaN,NaN,NaN,NaN,431.415923,446.290353
min,NaN,NaN,2024-07-04 16:50:04,18.000000,NaN,1.000000,5.200000,6.280000,NaN,NaN,...,0.000000,5.000000,1970-01-01 00:00:00.000000001,NaN,NaN,NaN,NaN,NaN,6.280000,13.570000
25%,NaN,NaN,2024-10-06 05:21:13.500000,33.000000,NaN,2.000000,60.145000,171.685000,NaN,NaN,...,0.000000,9.950000,1970-01-01 00:00:00.000000008,NaN,NaN,NaN,NaN,NaN,171.685000,186.552500
50%,NaN,NaN,2025-01-04 12:42:22.500000,48.000000,NaN,3.000000,112.465000,322.585000,NaN,NaN,...,0.000000,14.780000,1970-01-01 00:00:00.000000015,NaN,NaN,NaN,NaN,NaN,322.580000,337.510000
75%,NaN,NaN,2025-04-05 06:56:04,64.000000,NaN,4.000000,210.372500,623.160000,NaN,NaN,...,13.700000,19.765000,1970-01-01 00:00:00.000000022,NaN,NaN,NaN,NaN,NaN,623.137500,634.345000
max,NaN,NaN,2025-07-04 10:14:52,79.000000,NaN,19.000000,299.590000,1865.660000,NaN,NaN,...,50.000000,24.990000,1970-01-01 00:00:00.000000029,NaN,NaN,NaN,NaN,NaN,1865.610000,1883.390000



Distribución de customer_location:


customer_location
GB    325
MX    313
DE    312
CA    307
BR    304
JP    301
IN    301
AU    288
US    281
FR    268
Name: count, dtype: int64


Distribución de payment_method:


payment_method
PayPal            771
Credit Card       760
Apple Pay         554
Debit Card        550
Wire Transfer     194
Cryptocurrency    171
Name: count, dtype: int64


Distribución de shipping_speed:


shipping_speed
Standard    1099
Express      798
Priority     705
Economy      398
Name: count, dtype: int64

In [34]:
# ## 3) Filtros – `df_transacciones`
# Creamos **tres filtros** representativos:
# 1. **Transacciones de alto valor** (p. ej., top 10% por `total_amount`).
# 2. **Transacciones de riesgo**: si existe `velocity_flag`, `geolocation_mismatch` o `refund_requested`.
# 3. **Descuentos altos**: `discount_applied=True` y `discount_percentage >= 0.3` (30%) si esa columna existe.

In [35]:
def percentil(series, p=0.9):
    try:
        return series.quantile(p)
    except Exception:
        return None

filtros_transacciones = {}

if "total_amount" in df_transacciones.columns:
    umbral_top = percentil(df_transacciones["total_amount"], 0.9) or 0
    filtros_transacciones["alto_valor"] = df_transacciones[df_transacciones["total_amount"] >= umbral_top]

riesgo_cols = [c for c in ["velocity_flag","geolocation_mismatch","refund_requested"] if c in df_transacciones.columns]
if riesgo_cols:
    filtros_transacciones["riesgo_flags"] = df_transacciones[df_transacciones[riesgo_cols].fillna(False).any(axis=1)]

if all(c in df_transacciones.columns for c in ["discount_applied","discount_percentage"]):
    filtros_transacciones["descuento_alto"] = df_transacciones[
        (df_transacciones["discount_applied"] == True) &
        (df_transacciones["discount_percentage"].fillna(0) >= 0.30)
    ]

# Resumen de tamaños
for nombre, dfx in filtros_transacciones.items():
    print(f"{nombre}: {dfx.shape}")

alto_valor: (300, 22)
riesgo_flags: (1165, 22)
descuento_alto: (931, 22)


In [36]:
# ## 4) DataFrame de **Clientes** – `df_clientes`
# Agregamos por `customer_id`:
# - `total_pedidos`, `monto_total`, `ticket_promedio`
# - `tasa_reembolso` (si existe `refund_requested`)
# - `flags_riesgo` suma de (`velocity_flag`, `geolocation_mismatch`)
#
# Luego exploramos y generamos **tres filtros**:
# 1. **clientes_vip**: top 10% por `monto_total`
# 2. **clientes_riesgosos**: `tasa_reembolso >= 0.3` o `flags_riesgo > 0`
# 3. **clientes_frecuentes**: `total_pedidos >= 5`

In [37]:
from numpy import mean

def construir_df_clientes(df):
    req_cols = ["customer_id"]
    if not all(c in df.columns for c in req_cols):
        raise ValueError("No existe 'customer_id' en el dataset.")
    g = df.groupby("customer_id", dropna=False)

    out = pd.DataFrame({
        "total_pedidos": g.size(),
    })
    if "total_amount" in df.columns:
        out["monto_total"] = g["total_amount"].sum()
        out["ticket_promedio"] = g["total_amount"].mean()

    if "refund_requested" in df.columns:
        out["reembolsos"] = g["refund_requested"].sum()
        out["tasa_reembolso"] = (out["reembolsos"] / out["total_pedidos"]).round(3)
    else:
        out["tasa_reembolso"] = 0.0

    riesgo_sum = 0
    for flag in ["velocity_flag","geolocation_mismatch"]:
        if flag in df.columns:
            riesgo_sum = riesgo_sum + g[flag].sum()
    out["flags_riesgo"] = riesgo_sum if isinstance(riesgo_sum, pd.Series) else 0

    # trae alguna referencia demográfica si existe
    if "customer_location" in df.columns:
        out["customer_location_mas_comun"] = g["customer_location"].agg(lambda s: s.mode().iloc[0] if not s.mode().empty else None)

    out = out.reset_index()
    return out

df_clientes = construir_df_clientes(df_transacciones)

# Exploración
display(df_clientes.head(10))
display(df_clientes.describe(include="all"))

# Filtros (3)
filtros_clientes = {}
if "monto_total" in df_clientes.columns:
    umbral_vip = percentil(df_clientes["monto_total"], 0.9) or 0
    filtros_clientes["clientes_vip"] = df_clientes[df_clientes["monto_total"] >= umbral_vip]

filtros_clientes["clientes_frecuentes"] = df_clientes[df_clientes["total_pedidos"] >= 5]

if "tasa_reembolso" in df_clientes.columns:
    filtros_clientes["clientes_riesgosos"] = df_clientes[
        (df_clientes["tasa_reembolso"] >= 0.30) | (df_clientes["flags_riesgo"] > 0)
    ]

for nombre, dfx in filtros_clientes.items():
    print(f"{nombre}: {dfx.shape}")


,customer_id,total_pedidos,monto_total,ticket_promedio,reembolsos,tasa_reembolso,flags_riesgo,customer_location_mas_comun
0,CUST_10002,1,228.56,228.56,0,0.0,0,IN
1,CUST_10008,1,176.69,176.69,0,0.0,1,IN
2,CUST_10026,1,304.62,304.62,0,0.0,0,GB
3,CUST_10040,1,993.86,993.86,0,0.0,0,GB
4,CUST_10123,1,345.20,345.20,1,1.0,0,MX
5,CUST_10170,1,310.84,310.84,1,1.0,0,DE
6,CUST_10173,1,299.40,299.40,0,0.0,0,GB
7,CUST_10266,1,874.18,874.18,0,0.0,0,GB
8,CUST_10310,2,858.38,429.19,0,0.0,1,GB
9,CUST_10341,1,459.42,459.42,0,0.0,0,IN


,customer_id,total_pedidos,monto_total,ticket_promedio,reembolsos,tasa_reembolso,flags_riesgo,customer_location_mas_comun
count,2953,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953.000000,2953
unique,2953,NaN,NaN,NaN,NaN,NaN,NaN,10
top,CUST_10002,NaN,NaN,NaN,NaN,NaN,NaN,GB
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,320
mean,NaN,1.015916,438.281991,431.220619,0.182865,0.181172,0.273281,NaN
std,NaN,0.127850,338.760759,330.099076,0.386621,0.384125,0.488507,NaN
min,NaN,1.000000,6.280000,6.280000,0.000000,0.000000,0.000000,NaN
25%,NaN,1.000000,172.620000,172.030000,0.000000,0.000000,0.000000,NaN
50%,NaN,1.000000,330.240000,324.490000,0.000000,0.000000,0.000000,NaN
75%,NaN,1.000000,634.010000,623.310000,0.000000,0.000000,1.000000,NaN


clientes_vip: (296, 8)
clientes_frecuentes: (0, 8)
clientes_riesgosos: (1159, 8)


In [38]:
# ## 5) DataFrame de **Logística** – `df_logistica`
# A partir de columnas típicas (`shipping_speed`, `delivery_time_days`, `shipping_cost`):
# - Mapear un **SLA** (días objetivo por velocidad).
# - Calcular `cumple_sla` y analizar desempeño promedio por velocidad/envío.
#
# Filtros (3):
# 1. `envios_fuera_sla`: `cumple_sla == False`
# 2. `envios_costosos`: `shipping_cost` en top 10%
# 3. `envios_lentos`: `delivery_time_days >= 10`

In [39]:
def construir_df_logistica(df):
    req = ["shipping_speed","delivery_time_days","shipping_cost"]
    if not all(c in df.columns for c in req):
        raise ValueError("Faltan columnas logísticas (shipping_speed, delivery_time_days, shipping_cost).")

    df2 = df[req].copy()
    # normaliza tipos
    df2["delivery_time_days"] = pd.to_numeric(df2["delivery_time_days"], errors="coerce")
    df2["shipping_cost"] = pd.to_numeric(df2["shipping_cost"], errors="coerce")

    # SLA sugerido (ajusta según tu dominio)
    sla_map = {
        "standard": 7,
        "economy": 8,
        "express": 3,
        "two_day": 2,
        "overnight": 1
    }
    # normaliza etiquetas
    def norm(x):
        return str(x).strip().lower() if pd.notna(x) else x
    df2["shipping_speed_norm"] = df2["shipping_speed"].apply(norm)
    df2["sla_dias"] = df2["shipping_speed_norm"].map(sla_map).fillna(5)  # default 5 días
    df2["cumple_sla"] = df2["delivery_time_days"] <= df2["sla_dias"]

    # Agregación por velocidad
    agg = df2.groupby("shipping_speed_norm").agg(
        pedidos=("shipping_speed_norm","count"),
        costo_promedio=("shipping_cost","mean"),
        demora_promedio=("delivery_time_days","mean"),
        sla_dias=("sla_dias","first"),
        tasa_cumplimiento=("cumple_sla", "mean")
    ).reset_index()

    return df2, agg

df_logistica, df_logistica_resumen = construir_df_logistica(df_transacciones)

# Exploración
display(df_logistica.head(10))
display(df_logistica_resumen)

# Filtros (3)
filtros_logistica = {}
filtros_logistica["envios_fuera_sla"] = df_logistica[df_logistica["cumple_sla"] == False]

p90_costo = percentil(df_logistica["shipping_cost"].dropna(), 0.9) or 0
filtros_logistica["envios_costosos"] = df_logistica[df_logistica["shipping_cost"] >= p90_costo]

filtros_logistica["envios_lentos"] = df_logistica[df_logistica["delivery_time_days"] >= 10]

for nombre, dfx in filtros_logistica.items():
    print(f"{nombre}: {dfx.shape}")

,shipping_speed,delivery_time_days,shipping_cost,shipping_speed_norm,sla_dias,cumple_sla
0,Express,26,5.39,express,3.0,False
1,Express,25,8.18,express,3.0,False
2,Priority,27,8.61,priority,5.0,False
3,Standard,26,17.00,standard,7.0,False
4,Priority,1,20.53,priority,5.0,True
5,Express,4,17.06,express,3.0,False
6,Express,15,8.19,express,3.0,False
7,Express,14,12.74,express,3.0,False
8,Standard,13,7.93,standard,7.0,False
9,Standard,19,21.37,standard,7.0,False


,shipping_speed_norm,pedidos,costo_promedio,demora_promedio,sla_dias,tasa_cumplimiento
0,economy,398,15.125302,14.791457,8.0,0.271357
1,express,798,14.652343,15.337093,3.0,0.100251
2,priority,705,14.755277,14.814184,5.0,0.178723
3,standard,1099,15.022229,15.076433,7.0,0.228389


envios_fuera_sla: (2435, 6)
envios_costosos: (301, 6)
envios_lentos: (2088, 6)
